# QAFlow-style ETL & Analytics with Pandas (Test Runs, Orders, and Chat Logs)


In [2]:
import pandas as pd
import json
from pandas import json_normalize


## Test Data

In [3]:
test_runs = pd.DataFrame({
    "run_id": [1,2,3,4,5,6,7,8,9,10],
    "suite": ["login","login","checkout","checkout","checkout","profile","profile","login","profile","checkout"],
    "test_name": ["login_valid","login_invalid","checkout_pay","checkout_cancel","checkout_express",
                  "profile_update","profile_avatar","login_rate_limit","profile_privacy","checkout_discount"],
    "status": ["pass","fail","fail","pass","flaky","fail","pass","fail","flaky","pass"],
    "duration_s": [10.1, 8.5, 20.0, 15.2, 12.7, 9.9, 11.3, 10.5, 13.0, 18.4],
    "customer_id": [1001,1002,1001,1003,1001,1002,1003,1001,1002,1003]
})
test_runs.to_csv("test_runs.csv", index=False)


## Chat Data between Dev and QA engineer

In [4]:
chat_logs = [
    {
      "session_id": "s1",
      "messages": [
        {"speaker": "QA", "msg": "Test login failed, see stack trace", "ts": "2025-09-27T10:00:00"},
        {"speaker": "Dev", "msg": "Can you share the log?", "ts": "2025-09-27T10:01:00"},
        {"speaker": "QA", "msg": "Here is the log: …", "ts": "2025-09-27T10:02:00"}
      ]
    },
    {
      "session_id": "s2",
      "messages": [
        {"speaker": "QA", "msg": "Checkout discount test flaky", "ts": "2025-09-27T11:00:00"},
        {"speaker": "Dev", "msg": "I’ll investigate API call", "ts": "2025-09-27T11:01:00"}
      ]
    }
]
# You might save to file, or just keep as Python object:
with open("chat_logs.json", "w") as f:
    json.dump(chat_logs, f, indent=2)


## Orders Data

In [5]:
orders = pd.DataFrame({
    "customer_id": [1001, 1002, 1003, 1001, 1002],
    "price_usd": [500, 300, 450, 250, 150]
})
# (This would mirror “exported subscription / usage billing” data)


### Loading Test Data

In [6]:
df_runs=pd.read_csv('test_runs.csv')

In [7]:
df_runs.head()

,run_id,suite,test_name,status,duration_s,customer_id
0,1,login,login_valid,pass,10.1,1001
1,2,login,login_invalid,fail,8.5,1002
2,3,checkout,checkout_pay,fail,20.0,1001
3,4,checkout,checkout_cancel,pass,15.2,1003
4,5,checkout,checkout_express,flaky,12.7,1001


### Specifying failed or flaky test suite

In [8]:

df_ff = df_runs[df_runs["status"].isin(["fail", "flaky"])].copy()


In [9]:
df_ff

,run_id,suite,test_name,status,duration_s,customer_id
1,2,login,login_invalid,fail,8.5,1002
2,3,checkout,checkout_pay,fail,20.0,1001
4,5,checkout,checkout_express,flaky,12.7,1001
5,6,profile,profile_update,fail,9.9,1002
7,8,login,login_rate_limit,fail,10.5,1001
8,9,profile,profile_privacy,flaky,13.0,1002


### adding is_flaky flag

In [10]:
df_ff["is_flaky"] = df_ff["status"] == "flaky"


In [11]:
df_ff

,run_id,suite,test_name,status,duration_s,customer_id,is_flaky
1,2,login,login_invalid,fail,8.5,1002,False
2,3,checkout,checkout_pay,fail,20.0,1001,False
4,5,checkout,checkout_express,flaky,12.7,1001,True
5,6,profile,profile_update,fail,9.9,1002,False
7,8,login,login_rate_limit,fail,10.5,1001,False
8,9,profile,profile_privacy,flaky,13.0,1002,True


### adding metrics for failure rate (Transformation)

In [12]:
# Suite-level metrics
suite_stats = (
    #grou rows of suite by suite column
    df_ff.groupby("suite")
    .agg(
        #number of runs through run_id && count
        suite_n_runs=("run_id", "count"),
        #counts how many runs have status failed
        suite_n_fail=("status", lambda s: (s == "fail").sum()),
        #counts how many runs have status as flaky
        suite_n_flaky=("status", lambda s: (s == "flaky").sum()),
    )
    .reset_index()
)

In [13]:
# compute rates
suite_stats["suite_fail_rate"] = suite_stats["suite_n_fail"] / suite_stats["suite_n_runs"]
suite_stats["suite_flaky_rate"] = suite_stats["suite_n_flaky"] / suite_stats["suite_n_runs"]

In [14]:

# Merge back
df_ff = df_ff.merge(suite_stats, on="suite", how="left")


In [15]:
df_ff

,run_id,suite,test_name,status,duration_s,customer_id,is_flaky,suite_n_runs,suite_n_fail,suite_n_flaky,suite_fail_rate,suite_flaky_rate
0,2,login,login_invalid,fail,8.5,1002,False,2,2,0,1.0,0.0
1,3,checkout,checkout_pay,fail,20.0,1001,False,2,1,1,0.5,0.5
2,5,checkout,checkout_express,flaky,12.7,1001,True,2,1,1,0.5,0.5
3,6,profile,profile_update,fail,9.9,1002,False,2,1,1,0.5,0.5
4,8,login,login_rate_limit,fail,10.5,1001,False,2,2,0,1.0,0.0
5,9,profile,profile_privacy,flaky,13.0,1002,True,2,1,1,0.5,0.5


### Load

In [16]:
# 4. Load to new CSV
df_ff.to_csv("failed_flaky_runs.csv", index=False)


### B: Transforming Orders Data

In [17]:
### B. Orders aggregation

orders_summary = (
    orders.groupby("customer_id")["price_usd"]
    .agg(
        total_spend="sum",
        avg_spend="mean",
        n_orders="count",
        max_order="max"
    )
    .reset_index()
)

In [18]:
orders_summary

,customer_id,total_spend,avg_spend,n_orders,max_order
0,1001,750,375.0,2,500
1,1002,450,225.0,2,300
2,1003,450,450.0,1,450


### C: Transforming chat_logs Data

In [19]:
with open("chat_logs.json", "r") as f:
    chat_data = json.load(f)

chat_df = json_normalize(chat_data, record_path="messages", meta=["session_id"])
# chat_df columns: session_id, speaker, msg, ts

In [20]:
# Compute session-level summary
def count_turns(speakers: list):
    # count number of times speaker changes
    turns = 0
    prev = None
    for sp in speakers:
        if prev is not None and sp != prev:
            turns += 1
        prev = sp
    return turns


In [21]:
chat_summary = (
    chat_df.groupby("session_id")
    .agg(
        n_messages=("msg", "count"),
        n_qa_msgs=("speaker", lambda s: (s == "QA").sum()),
        n_dev_msgs=("speaker", lambda s: (s == "Dev").sum()),
        avg_msg_len=("msg", lambda msgs: msgs.str.len().mean()),
        speakers_list=("speaker", lambda s: list(s))
    )
    .reset_index()
)

In [22]:
chat_summary

,session_id,n_messages,n_qa_msgs,n_dev_msgs,avg_msg_len,speakers_list
0,s1,3,2,1,24.666667,"[QA, Dev, QA]"
1,s2,2,1,1,26.500000,"[QA, Dev]"


In [23]:
chat_summary["turns"] = chat_summary["speakers_list"].apply(count_turns)
# Optionally drop the speakers_list column
chat_summary = chat_summary.drop(columns=["speakers_list"])


In [24]:
chat_summary

,session_id,n_messages,n_qa_msgs,n_dev_msgs,avg_msg_len,turns
0,s1,3,2,1,24.666667,2
1,s2,2,1,1,26.500000,1


### D: Summarizing Information from the transformed Data

In [25]:
### D. summary function

def generate_summary(suite_stats: pd.DataFrame, orders_summary: pd.DataFrame, chat_summary: pd.DataFrame) -> str:
    # Top suites by fail_rate
    top = suite_stats.sort_values("suite_fail_rate", ascending=False).head(3)
    lines = []
    lines.append("Test Suites with highest failure rates:")
    for _, row in top.iterrows():
        lines.append(f" - suite {row['suite']}: fail_rate={row['suite_fail_rate']:.2f}, flaky_rate={row['suite_flaky_rate']:.2f}")

    lines.append("\nTop-spending customers:")
    cs = orders_summary.sort_values("total_spend", ascending=False).head(3)
    for _, r in cs.iterrows():
        lines.append(f" - {int(r['customer_id'])} with total_spend {r['total_spend']:.0f} USD over {r['n_orders']} orders")

    lines.append("\nChat sessions analysis:")
    for _, r in chat_summary.iterrows():
        lines.append(
            f" - session {r['session_id']} had {r['n_messages']} messages ({int(r['n_qa_msgs'])} QA / {int(r['n_dev_msgs'])} Dev), "
            f"average length {r['avg_msg_len']:.1f} chars, {r['turns']} turns"
        )

    return "\n".join(lines)

In [27]:
summary_text = generate_summary(suite_stats, orders_summary, chat_summary)
print(summary_text)


Test Suites with highest failure rates:
 - suite login: fail_rate=1.00, flaky_rate=0.00
 - suite checkout: fail_rate=0.50, flaky_rate=0.50
 - suite profile: fail_rate=0.50, flaky_rate=0.50

Top-spending customers:
 - 1001 with total_spend 750 USD over 2.0 orders
 - 1002 with total_spend 450 USD over 2.0 orders
 - 1003 with total_spend 450 USD over 1.0 orders

Chat sessions analysis:
 - session s1 had 3 messages (2 QA / 1 Dev), average length 24.7 chars, 2 turns
 - session s2 had 2 messages (1 QA / 1 Dev), average length 26.5 chars, 1 turns


In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model + tokenizer
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


d:\venvs\Agents-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  Fil

In [ ]:
import os
from huggingface_hub import InferenceClient
from dotenv import load_dotenv

# 1️⃣ Load environment variables
load_dotenv()
hf_token = os.getenv("HF_TOKEN")   # Make sure your .env file contains: HF_TOKEN=your_token_here
if not hf_token:
    raise ValueError("HF_TOKEN not found in .env file")

# 2️⃣ Initialize Hugging Face inference client
client = InferenceClient(token=hf_token)

# 3️⃣ Choose a model (instruction-tuned recommended)
model_name = "facebook/bart-large-cnn"   # You can also try "mistralai/Mistral-7B-Instruct-v0.2"

def generate_llm_summary(
    suite_stats: pd.DataFrame,
    orders_summary: pd.DataFrame,
    chat_summary: pd.DataFrame
) -> str:
    """
    Convert analytics data into a textual prompt for the LLM
    and return a concise summary.
    """
    prompt_lines = ["You are a QA analyst assistant. Analyze the following QA data:\n"]

    # Test Suite Stats
    prompt_lines.append("Test Suite Stats:")
    for _, row in suite_stats.iterrows():
        prompt_lines.append(
            f"- Suite {row['suite']}: {row['suite_n_runs']} runs, "
            f"fail_rate={row['suite_fail_rate']:.2f}, flaky_rate={row['suite_flaky_rate']:.2f}"
        )

    # Customer Orders
    prompt_lines.append("\nCustomer Orders:")
    for _, row in orders_summary.iterrows():
        prompt_lines.append(
            f"- Customer {int(row['customer_id'])}: total_spend={row['total_spend']} USD, "
            f"avg={row['avg_spend']:.2f} USD, n_orders={row['n_orders']}"
        )

    # Chat Sessions
    prompt_lines.append("\nChat Sessions:")
    for _, row in chat_summary.iterrows():
        prompt_lines.append(
            f"- Session {row['session_id']}: {row['n_messages']} messages "
            f"({int(row['n_qa_msgs'])} QA / {int(row['n_dev_msgs'])} Dev), "
            f"avg_len={row['avg_msg_len']:.1f} chars, turns={row['turns']}"
        )

    # Instruction
    prompt_lines.append(
        "\nGenerate a concise QA report highlighting the riskiest test suites, "
        "top-spending customers, and sessions needing attention."
    )

    prompt_text = "\n".join(prompt_lines)

    response = client.text_generation(
        model="tiiuae/falcon-7b-instruct",
        inputs=prompt_text,
        parameters={
            "max_new_tokens": 300,  # Adjust based on desired summary length
            "temperature": 0.7,     # Controls randomness; 0.7 is a balanced choice
            "top_p": 0.9,           # Nucleus sampling; 0.9 ensures diversity
            "top_k": 50,            # Limits sampling to top 50 tokens
            "repetition_penalty": 1.2,  # Discourages repetitive phrases
            "stop": ["\n"]          # Stops generation at newline
        }
    )

    # `response` is a list of dicts; extract generated text
    summary_text = response
    return summary_text


In [115]:
summary = generate_llm_summary(suite_stats, orders_summary, chat_summary)
print("=== LLM Summary ===")
print(summary)


=== LLM Summary ===
SummarizationOutput(summary_text='Generate a concise QA report highlighting the riskiest test suites, top-spending customers, and sessions needing attention. You are a QA analyst assistant. Analyze the following QA data: Test Suite Stats: Suite checkout: 2 runs, fail_rate=0.50, flaky_rate =0. 50. Suite login: 2 Runs, fail-rate=1.00, flaky-rate: 0.00. Suite profile: 2 Running Runs, failed-rate = 0.50. Chat Sessions: Session s1: 3 messages (2 QA / 1 Dev), avg_len=24.7 chars, turns=2. Session s')


In [101]:
def ask_llm_question(summary_text, question):
    prompt = f"{summary_text}\n\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(**inputs, max_length=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

answer = ask_llm_question(summary, "Which test suite should we fix first?")
print(answer)


AttributeError: 'tuple' object has no attribute 'generate'

## Conclusion

This notebook demonstrated a **full ETL + analytics workflow** on Mocked QAFlow-style data:

- **Test Runs**: filtered failed/flaky tests, engineered features, and computed suite-level failure metrics.
- **Customer Orders**: aggregated spend, order counts,etc.
- **Chat Logs**: normalized nested JSON and derived conversation statistics.
- **Reporting**: generated a concise, LLM-ready summary for quick insights.  

These steps showcase how **pandas** can power data-engineering tasks relevant to QA automation, analytics, and LLM-driven quality platforms.

*Prepared by **Mohamed Hafez***
